In [1]:
!pip install contractions

You should consider upgrading via the 'c:\users\quang binh\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import os, re
import numpy as np
import pandas as pd
np.random.seed(2020)

import nltk
nltk.download('all') # one time execution
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prepare Data


In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/data_final.csv')
data.head()

In [ ]:
data.dropna()
data = data.drop(columns=['Unnamed: 0'])
data.info()

In [ ]:
data = data.rename(columns = {"song name": "song_name",
                              "song lyric": "song_lyric"})
data = data.astype(np.str)
data.head()

# Text Preprocessing

In [ ]:
import pandas as pd
import re
import string
import contractions
contractions_dict = {"ain't": "are not",
                     "'s":" is",
                     "aren't": "are not",
                     "'fore": "before", 
                     "i've": "i have", 
                     "you've": "you have",
                     "wanna": "want to", 
                     "gotta": "have got to",
                     "gonna": "going to",
                     "ima": "i am going to",
                     "you'll": "you will",
                     "i'll": "i will"}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

In [ ]:
class TextPreprocessor:

    def __init__(self, dataframe):    
        self.df = dataframe

    # delete punctuations
    def remove_punctuation(self, text):       
        return re.sub(f'[{string.punctuation}]', '', text)

    # lower texts
    def to_lowercase(self, text):       
        return text.lower()
    
     # remove numbers
    def remove_numbers(self, text):     
        return re.sub(r'\d+', '', text)
    
     # remove extra white spaces
    def remove_whitespaces(self, text):      
        return text.strip()
    
     # expand contradiction
    def expand_contraction(self, text, contractions_dict = contractions_dict):     
        expanded_words = []   
        for word in text.split():
            expanded_words.append(contractions.fix(word))  
        expanded_text = ' '.join(expanded_words)

        def replace(match):
            return contractions_dict[match.group(0)]
        return contractions_re.sub(replace, expanded_text)
    
     # delete non-english words
    def del_nonEnglish(self, text):      
        text = re.sub(r'\W+', ' ', text)
        text = text.lower()
        text = text.replace("[^a-zA-Z]", " ")
        word_tokens = word_tokenize(text)
        filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
        filtered_word = [w + " " for w in filtered_word]
        return "".join(filtered_word)
    
     # remove stopwords
    def remove_stopwords(self, text):    
        global stop_words
        try:
            word_tokens = word_tokenize(text)
            filtered_word = [w for w in word_tokens if not w in stop_words]
            filtered_word = [w + " " for w in filtered_word]
            return "".join(filtered_word)
        except:
            return np.nan

     #lemmatize
    def normalization(self, text):      
        global lemmatizer
        return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    
    def preprocess_text(self):
        self.df['name_pp'] = self.df.song_name.apply(self.to_lowercase)
        self.df['name_pp'] = self.df.name_pp.apply(self.expand_contraction)
        self.df['name_pp'] = self.df.name_pp.apply(self.remove_punctuation)
        self.df['name_pp'] = self.df.name_pp.apply(self.del_nonEnglish)
        self.df['name_pp'] = self.df.name_pp.apply(self.remove_stopwords)
        self.df['name_pp'] = self.df.name_pp.apply(self.normalization)
        self.df['name_pp'] = self.df.name_pp.apply(self.remove_whitespaces)
        self.df['name_pp'] = self.df.name_pp.apply(self.remove_numbers)

        self.df['lyric_pp'] = self.df.song_lyric.apply(self.to_lowercase)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.expand_contraction)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.remove_punctuation)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.del_nonEnglish)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.remove_stopwords)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.normalization)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.remove_whitespaces)
        self.df['lyric_pp'] = self.df.lyric_pp.apply(self.remove_numbers)
        
        return self.df


In [ ]:
data = TextPreprocessor(data).preprocess_text()

In [ ]:
data = data.drop(columns=['song_name', 'song_lyric'])

In [ ]:
data.to_csv(r'/content/drive/MyDrive/data_final_preprocessed.csv')